# Market Intelligence Analysis

This notebook analyzes Hacker News job data to uncover market trends, salary insights, and remote work patterns.
It focuses on understanding the data structure and the distribution of extracted features, including seniority, location tiers, company stage, and compensation structures.

## 1. Setup and Data Loading

In [53]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

# Load the structured data
df = pd.read_parquet('../../data/hn_jobs_structured.parquet')

# Display the first few rows and info
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 37115 entries, 0 to 37563
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     37115 non-null  object        
 1   date                   37115 non-null  datetime64[ns]
 2   raw_text               37115 non-null  object        
 3   company_name           34381 non-null  object        
 4   role_title             34847 non-null  object        
 5   salary_min             5865 non-null   Int64         
 6   salary_max             5865 non-null   Int64         
 7   salary_avg             5865 non-null   Int64         
 8   currency               5865 non-null   object        
 9   is_remote              37115 non-null  bool          
 10  tech_stack             37115 non-null  object        
 11  job_category           37115 non-null  object        
 12  is_senior              37115 non-null  bool          
 13  is_jun

,id,date,raw_text,company_name,role_title,salary_min,salary_max,salary_avg,currency,is_remote,...,is_tier_1_city,is_europe,is_global_remote,is_yc,is_funded,is_crypto,has_equity,offers_visa,tech_combo_ai,tech_combo_blockchain
0,21937030,2020-01-01,"CodeWeavers | St Paul, MN, USA | Full Time | R...",CodeWeavers,"St Paul, MN, USA",<NA>,<NA>,<NA>,None,True,...,False,False,False,False,False,False,False,True,False,False
1,21937183,2020-01-01,GiveCampus (YC S15) | software engineers | Ful...,GiveCampus (YC S15),software engineers,<NA>,<NA>,<NA>,None,False,...,False,False,False,True,True,False,True,False,False,False
2,21937418,2020-01-01,"Citymapper | Full-time, VISA and Remote (for e...",Citymapper,"Full-time, VISA and Remote (for experienced ca...",<NA>,<NA>,<NA>,None,True,...,True,True,False,False,False,False,False,True,False,False
3,21937031,2020-01-01,Vitally.io | Senior Backend/API Engineer (Node...,Vitally.io,Senior Backend/API Engineer (Node + Typescript),<NA>,<NA>,<NA>,None,False,...,True,False,False,False,True,False,False,False,False,False
4,21942465,2020-01-01,"Composable Analytics | Boston, MA | \nhttps://...",Composable Analytics,"Boston, MA",<NA>,<NA>,<NA>,None,False,...,False,False,False,False,False,False,False,False,False,False


## 2. Data Sanity Check and Cleaning

Inspect the `salary_avg` column and filter out unrealistic values.

In [54]:
# Check salary statistics
print("Before cleaning:")
print(df['salary_avg'].describe())

# 1. Fix Monthly Salaries (e.g., $6k -> $72k)
# Assumption: Salaries between $2k and $15k are likely monthly.
monthly_mask = (df['salary_avg'] >= 2000) & (df['salary_avg'] <= 15000)
print(f"\nConverting {monthly_mask.sum()} rows from monthly to annual salary.")
df.loc[monthly_mask, 'salary_avg'] = df.loc[monthly_mask, 'salary_avg'] * 12

# 2. Filter Outliers
# Low end: < $15k (after monthly fix)
# High end: > $500k (likely errors or extreme outliers)
low_salary_mask = (df['salary_avg'] < 15000) & (df['salary_avg'].notna())
high_salary_mask = (df['salary_avg'] > 500000) & (df['salary_avg'].notna())

print(f"Dropping {low_salary_mask.sum()} rows with salary < $15k")
print(f"Dropping {high_salary_mask.sum()} rows with salary > $500k")

# Apply filter
df = df[~low_salary_mask & ~high_salary_mask]

print("\nAfter cleaning:")
print(df['salary_avg'].describe())

Before cleaning:
count           5865.0
mean     150126.834101
std      166375.289711
min              123.0
25%            56575.0
50%           120000.0
75%           175000.0
max          1080000.0
Name: salary_avg, dtype: Float64

Converting 116 rows from monthly to annual salary.
Dropping 727 rows with salary < $15k
Dropping 275 rows with salary > $500k

After cleaning:
count           4863.0
mean     142363.653095
std       89340.181238
min            15200.0
25%            85000.0
50%           135000.0
75%           175000.0
max           500000.0
Name: salary_avg, dtype: Float64


## 3. Feature Distribution Overview

Let's explore the distribution of the newly extracted features to understand the composition of the dataset.

In [63]:
# List all columns to verify new features are present
print("Columns in dataset:", df.columns.tolist())

# Define feature groups
feature_groups = {
    "Seniority": ['is_senior', 'is_junior', 'is_manager'],
    "Location Tiers": ['is_tier_1_city', 'is_europe', 'is_global_remote'],
    "Company Stage": ['is_yc', 'is_funded', 'is_crypto'],
    "Compensation": ['has_equity', 'offers_visa'],
    "Interactions": ['tech_combo_ai', 'tech_combo_blockchain']
}

# Calculate and plot prevalence of each feature
feature_stats = []
for group, features in feature_groups.items():
    for feature in features:
        if feature in df.columns:
            count = df[feature].sum()
            pct = (count / len(df)) * 100
            feature_stats.append({'Group': group, 'Feature': feature, 'Count': count, 'Percentage': pct})

stats_df = pd.DataFrame(feature_stats)

# Display the stats
print("\nFeature Prevalence:")
print(stats_df.sort_values(['Group', 'Percentage'], ascending=[True, False]).to_string(index=False))

# Visualize Feature Prevalence
fig = px.bar(stats_df, x='Percentage', y='Feature', color='Group', orientation='h',
             title='Prevalence of Extracted Features (% of Job Posts)',
             text='Percentage')
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(height=600)
fig.show()

# Years of Experience Distribution
if 'years_experience' in df.columns:
    # Cap years of experience at 30 to remove outliers
    outlier_count = len(df[df['years_experience'] > 30])
    if outlier_count > 0:
        print(f"\nCapping {outlier_count} rows with > 30 years experience to 30.")
        df.loc[df['years_experience'] > 30, 'years_experience'] = 30
    
    print("\nYears of Experience Statistics:")
    print(df['years_experience'].describe())
    
    fig_exp = px.histogram(df, x='years_experience', nbins=30, 
                           title='Distribution of Required Years of Experience (Capped at 30)',
                           labels={'years_experience': 'Years of Experience'})
    fig_exp.show()
else:
    print("\n'years_experience' column not found.")

Columns in dataset: ['id', 'date', 'raw_text', 'company_name', 'role_title', 'salary_min', 'salary_max', 'salary_avg', 'currency', 'is_remote', 'tech_stack', 'job_category', 'is_senior', 'is_junior', 'is_manager', 'years_experience', 'is_tier_1_city', 'is_europe', 'is_global_remote', 'is_yc', 'is_funded', 'is_crypto', 'has_equity', 'offers_visa', 'tech_combo_ai', 'tech_combo_blockchain', 'month']

Feature Prevalence:
         Group               Feature  Count  Percentage
 Company Stage             is_funded   6587   18.239969
 Company Stage                 is_yc   6015   16.656052
 Company Stage             is_crypto   3310    9.165674
  Compensation            has_equity   8342   23.099715
  Compensation           offers_visa   2904    8.041426
  Interactions         tech_combo_ai    650    1.799906
  Interactions tech_combo_blockchain    462    1.279318
Location Tiers        is_tier_1_city  13976   38.700745
Location Tiers             is_europe   6412   17.755379
Location Tiers     


Years of Experience Statistics:
count      3625.0
mean     7.144276
std      6.899338
min           1.0
25%           3.0
50%           5.0
75%           8.0
max          30.0
Name: years_experience, dtype: Float64


## 4. Data Quality Check: Raw Text vs Extracted Features

Let's look at a few random samples to verify if the extraction logic seems to be working as expected by comparing the `raw_text` with the extracted flags.

In [56]:
# Sample a few rows to inspect
sample_cols = ['raw_text', 'role_title', 'salary_avg'] + [f for f in stats_df['Feature'].tolist()]
sample_df = df[sample_cols].sample(5)

for idx, row in sample_df.iterrows():
    print(f"--- Job ID: {idx} ---")
    print(f"Raw Text: {row['raw_text'][:200]}...") # Truncate for display
    print("Extracted Features:")
    for feature in stats_df['Feature'].tolist():
        if row[feature]:
            print(f"  - {feature}")
    print(f"  - Role: {row['role_title']}")
    print(f"  - Salary: ${row['salary_avg']:,.0f}" if pd.notna(row['salary_avg']) else "  - Salary: N/A")
    print("\n")

--- Job ID: 21360 ---
Raw Text: American Eagle | Remote | Full-time | Multiple positions
We are a startup inside American Eagle and we are building the future of supply chain and logistics through enterprise crowd sourcing. This art...
Extracted Features:
  - is_manager
  - Role: Remote
  - Salary: N/A


--- Job ID: 4515 ---
Raw Text: MeetKai | Full-Time / Intern | REMOTE | Los Angeles / US / China | 60-180K + Equity for qualified candidates
We are a stealth-ish startup in the conversational AI space, co-founded by a founder of a >...
Extracted Features:
  - is_junior
  - is_manager
  - is_global_remote
  - has_equity
  - Role: Full-Time / Intern
  - Salary: $120,000


--- Job ID: 8461 ---
Raw Text: HealthiVibe | Senior Backend Developer - Clojure | London, UK | Remote (UK Only) | Full Time
HealthiVibe, a division of Corrona LLC, is a market leader in innovative, evidence-based patient engagement...
Extracted Features:
  - is_senior
  - is_tier_1_city
  - is_europe
  - is_yc
  - Role: 

## 5. Data Preparation: Exploding the Tech Stack

Transform the dataset to have one row per technology.

In [57]:
# Explode the tech_stack column
df_exploded = df.explode('tech_stack')

# Rename the column for clarity
df_exploded = df_exploded.rename(columns={'tech_stack': 'tech'})

# Drop rows where tech is NaN (if any)
df_exploded = df_exploded.dropna(subset=['tech'])

print(f"Original rows: {len(df)}")
print(f"Exploded rows: {len(df_exploded)}")
df_exploded.head()

Original rows: 36113
Exploded rows: 94005


,id,date,raw_text,company_name,role_title,salary_min,salary_max,salary_avg,currency,is_remote,...,is_tier_1_city,is_europe,is_global_remote,is_yc,is_funded,is_crypto,has_equity,offers_visa,tech_combo_ai,tech_combo_blockchain
0,21937030,2020-01-01,"CodeWeavers | St Paul, MN, USA | Full Time | R...",CodeWeavers,"St Paul, MN, USA",<NA>,<NA>,<NA>,None,True,...,False,False,False,False,False,False,False,True,False,False
1,21937183,2020-01-01,GiveCampus (YC S15) | software engineers | Ful...,GiveCampus (YC S15),software engineers,<NA>,<NA>,<NA>,None,False,...,False,False,False,True,True,False,True,False,False,False
1,21937183,2020-01-01,GiveCampus (YC S15) | software engineers | Ful...,GiveCampus (YC S15),software engineers,<NA>,<NA>,<NA>,None,False,...,False,False,False,True,True,False,True,False,False,False
1,21937183,2020-01-01,GiveCampus (YC S15) | software engineers | Ful...,GiveCampus (YC S15),software engineers,<NA>,<NA>,<NA>,None,False,...,False,False,False,True,True,False,True,False,False,False
1,21937183,2020-01-01,GiveCampus (YC S15) | software engineers | Ful...,GiveCampus (YC S15),software engineers,<NA>,<NA>,<NA>,None,False,...,False,False,False,True,True,False,True,False,False,False


## 6. Module A: Trend Analysis - "The Rise and Fall of Tech"

Question: Which technologies are winning the startup war?
Metric: Percentage of job posts mentioning a specific technology per month.

In [58]:
# Convert date to datetime if not already
df_exploded['date'] = pd.to_datetime(df_exploded['date'])
df_exploded['month'] = df_exploded['date'].dt.to_period('M').astype(str)

# Calculate total jobs per month
# Use the original df for total jobs to avoid double counting exploded rows
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M').astype(str)
jobs_per_month = df.groupby('month')['id'].count().reset_index()
jobs_per_month.columns = ['month', 'total_jobs']

# Calculate tech counts per month
tech_per_month = df_exploded.groupby(['month', 'tech'])['id'].count().reset_index()
tech_per_month.columns = ['month', 'tech', 'tech_count']

# Merge to calculate percentage
trend_df = pd.merge(tech_per_month, jobs_per_month, on='month')
trend_df['percentage'] = (trend_df['tech_count'] / trend_df['total_jobs']) * 100

# Select competitors to compare
competitors = ['Python', 'JavaScript', 'TypeScript', 'Rust', 'Go', 'Java', 'C++']

subset_trend = trend_df[trend_df['tech'].isin(competitors)]

# Plot
fig = px.line(subset_trend, x='month', y='percentage', color='tech',
              title='Tech Trends: % of Job Posts Mentioning Technology',
              labels={'percentage': '% Frequency', 'month': 'Date'},
              markers=True)
fig.show()

## 7. Module B: Salary Analysis - "The Value of Skills"

Question: Does knowing Rust actually pay more than Python?
Metric: Median Salary (Annual USD) per Technology.

In [59]:
# Filter for non-null salary
df_salary = df_exploded[df_exploded['salary_avg'].notna()]

# Get top 10 technologies by popularity (count)
top_tech = df_salary['tech'].value_counts().nlargest(10).index.tolist()

# Filter data for top 10 tech
df_salary_top = df_salary[df_salary['tech'].isin(top_tech)]

# Sort by median salary
median_salary = df_salary_top.groupby('tech')['salary_avg'].median().sort_values(ascending=False)
order = median_salary.index.tolist()

# Plot
fig = px.box(df_salary_top, x='tech', y='salary_avg',
             category_orders={'tech': order},
             title='Salary Distribution by Technology (Top 10 Popular)',
             labels={'salary_avg': 'Annual Salary ($)', 'tech': 'Technology'},
             color='tech')
fig.show()

## 8. Module C: The "Remote" Reality Check

Question: Are companies forcing people back to the office?
Metric: Percentage of "Remote" vs "Onsite" jobs over time.

In [60]:
# Check if 'is_remote' column exists and its values
remote_col = 'is_remote'
if remote_col in df.columns:
    print(f"Dataframe shape: {df.shape}")
    print(f"Remote column nulls: {df[remote_col].isna().sum()}")
    print(df[remote_col].value_counts(dropna=False))
    
    # Prepare data for stacked area chart
    # Group by month and remote status
    # Ensure month column exists in df
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.to_period('M').astype(str)
    
    remote_counts = df.groupby(['month', remote_col]).size().reset_index(name='count')
    
    # Calculate percentage per month
    monthly_totals = remote_counts.groupby('month')['count'].transform('sum')
    remote_counts['percentage'] = (remote_counts['count'] / monthly_totals) * 100
    
    # Plot
    fig = px.area(remote_counts, x='month', y='percentage', color=remote_col,
                  title='Remote vs Onsite Jobs Over Time',
                  labels={'percentage': '% of Jobs', 'month': 'Date'},
                  groupnorm='percent') # This ensures it's 100% stacked
    fig.show()
else:
    print(f"Column '{remote_col}' not found in dataframe.")

Dataframe shape: (36113, 27)
Remote column nulls: 0
is_remote
True     25719
False    10394
Name: count, dtype: int64
